In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import roc_curve, roc_auc_score
from Helpers import *
from functools import reduce
from glob import glob
from tqdm import tqdm

In [2]:
data_directories = ['./Multi-Populated-Unfiltered/', './Multi-Populated-Unfiltered-NoPseudo/', 
                    './Multi-Populated-Unfiltered-Ensemble/', './Multi-Populated-Unfiltered-NoPseudo-Ensemble/'] 
data_directories = ['./Data-50/', './Data-50-NoPseudo/',
                   './Data-50-Ensemble/', './Data-50-NoPseudo-Ensemble/'] 
model_type = 'DNN'

In [3]:
for data_dir in tqdm(data_directories):
    save_preds = '{}{}-preds.csv'.format(data_dir, model_type)

    df_predictions = None
    file_pointers = glob('{}*.pickle'.format(data_dir))
    
    for data_fp in tqdm(file_pointers):
        model_nam, dataset, clss = data_fp.split('/')[-1].split('_')
        clss = clss.split('.')[0]

        with open(data_fp, 'rb') as fp:
            raw_data = pickle.load(fp)
        predict_pos, predict_neg, fold_preds, fold_labs, lr, batch_size  = raw_data.data

        ## Store Fold Predictions
        tmp_predictions = None
        for i, fold_set in enumerate(fold_preds):
            for j, (pred_pos, _) in enumerate(fold_set):
                if pred_pos:
                    predictions, labels = fold_labs[i][j]
                    tmp_predictions = pd.concat((pd.DataFrame({'Score': predictions, 'Label': labels, 'Fold_Set': i, 'Fold': j}), 
                                                              tmp_predictions))
        tmp_predictions['Model'] = model_nam
        tmp_predictions['Class'] = clss
        tmp_predictions['Dataset'] = dataset
        df_predictions = pd.concat((df_predictions, tmp_predictions))
        
    df_predictions.to_csv(save_preds, index=False)
    print('Saved {}'.format(data_dir))

  6%|▋         | 1/16 [00:00<00:02,  5.18it/s]

Saved ./Data-50/



  6%|▋         | 1/16 [00:00<00:02,  5.13it/s]

Saved ./Data-50-NoPseudo/



  6%|▋         | 1/16 [00:00<00:02,  5.21it/s]

Saved ./Data-50-Ensemble/



100%|██████████| 4/4 [00:13<00:00,  3.47s/it]

Saved ./Data-50-NoPseudo-Ensemble/
